# Notebook para contar archivos en un directorio y ordenarlos por tamaño y sacar estadísticas

In [5]:
#Primero importamos las librerías necesarias y definimos las funciones que usaremos
import os
import pandas as pd

#Función para convertir a MB,GB, etc.
def convert_bytes(num):
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        #print(x,num)
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0
        
#Necesitamos definir la ruta que queremos analizar
#path = "F:\DATA_TFM\CSVs\\"
path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\"

#Después, generamos la lista con los archivos a analizar
arr = os.listdir(path); arr = sorted(arr)

#Esta funcion obtiene los nombres de los archivo y tamaños usando series de pandas
def gen_lista_series(path,arr):
    columnas = ["file", "size"]
    lista_df = pd.DataFrame(columns=columnas)
    #Recorremos cada archivo para obtener su tamaño, convertimos a series y añadimos al DF
    for i in range(len(arr)):
        file = path + arr[i]; info = os.stat(file)
        file_size = info.st_size
        temp = [arr[i], file_size]
        temp = pd.Series(temp,index=columnas)
        lista_df=lista_df.append(temp,ignore_index=True)
    return(lista_df)

#Función para calcular cuantos archivos faltan y tamaño.
def calcula_faltante(archivo,df):
    #Buscamos el numero de indice y lo convrt. a entero
    ind = df[df['file']==archivo].index.tolist()
    ind = ind[0]
    tempdf = df.iloc[ind:]
    tempdf = tempdf['size']
    print("Te faltan:", tempdf.shape[0],"archivos")
    print("Y eso da: ", convert_bytes(tempdf.sum()))
    
#Otra forma de hacerlo, usando listas y creando el df hasta el final
#Esta forma es mucho más eficiente en tiempo (muchísimo)
def gen_lista_lists(path,arr):
    columnas = ["file", "size"]
    lista_df = pd.DataFrame(columns=columnas)
    temp = []
    #Recorremos cada archivo para obtener su tamaño, convertimos a series y añadimos al DF
    for i in range(len(arr)):
        file = path + arr[i]; info = os.stat(file)
        file_size = info.st_size
        elem = [arr[i], file_size]
        temp.append(elem)   
    lista_df=pd.DataFrame(temp, columns=columnas)
    return(lista_df)

In [13]:
#Calculamos cuanto falta para que termine (archivos y tamaño)
archivo = "MC_045780_numu_CC_a_reco.i3.csv"
lista_df = gen_lista_lists(path,arr)

calcula_faltante(archivo,lista_df)

Te faltan: 423 archivos
Y eso da:  18.6 GB


In [3]:
### Comparando el tiempo que tardan ambas funciones

import timeit

%timeit gen_lista_series(path,arr)

%timeit gen_lista_lists(path,arr)

###### SE PUEDE VER QUE EL UTILIZAR LISTAS Y CREAR EL DATAFRAME HASTA EL FINAL ES ORDENES DE MAGNITUD MEJOR 100 VECES

4.21 s ± 184 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
77.3 ms ± 801 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
